Summary: 
This script implements a production ready ETL pipeline for gaming data analysis. It aims to produce aggregated performance metrics by demographic and gaming attributes.


Hint for In df_casinomanufacturers !
Here we spotted 6 null values in the column LatestFlag, however in the csv file we don't have any missing value in this column. So, after exploring the rows that presented missing values we understood that the csv file in the rows 19, 72, 73, 145, 271 had " ", which cofused pandas while reading the file. 
Thus, we fix this issue in the beggining of the script.

# Load the libraries - Data

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import date
import seaborn as sns 


# Data from CSVs & Read csvs using pandas

casinodaily = r"\Users\dimitra.chatzopoulou\Desktop\demi\pesonal\CV\novibet\casino_BI_Engineer\casinodaily.csv"
casinomanufacturers = r"\Users\dimitra.chatzopoulou\Desktop\demi\pesonal\CV\novibet\casino_BI_Engineer\casinomanufacturers.csv"
casinoproviders = r"\Users\dimitra.chatzopoulou\Desktop\demi\pesonal\CV\novibet\casino_BI_Engineer\casinoproviders.csv"
currencyrates = r"\Users\dimitra.chatzopoulou\Desktop\demi\pesonal\CV\novibet\casino_BI_Engineer\currencyrates.csv"
users = r"\Users\dimitra.chatzopoulou\Desktop\demi\pesonal\CV\novibet\casino_BI_Engineer\users.csv"

df_casinodaily = pd.read_csv(casinodaily, header=0, delimiter=',')
df_casinoproviders = pd.read_csv(casinoproviders, header=0, delimiter=',')
df_currencyrates = pd.read_csv(currencyrates, header=0, delimiter=',')
df_users = pd.read_csv(users, header=0, delimiter=',')

# For df_casinomanufacturers

# set the column names
col_names = ["CasinoManufacturerId",
    "CasinoManufacturerName",
    "FromDate",
    "ToDate",
    "LatestFlag"]
df_casinomanufacturers = pd.read_csv(
    casinomanufacturers,
    skiprows=1,       # skip the bad header row
    header=None,      # so pandas doesn’t try to interpret row 0 as column names
    names=col_names)  # assign these exact five names
null_mask = df_casinomanufacturers['LatestFlag'].isnull()
df_casinomanufacturers = df_casinomanufacturers.replace('"', '', regex=True) # Remove all double quotes from the entire df
# Fix each problematic row
for idx in df_casinomanufacturers[null_mask].index:
    checkdata = str(df_casinomanufacturers.loc[idx, 'CasinoManufacturerId'])
    parts = checkdata.split(',')
    if len(parts) >= 5:
        df_casinomanufacturers.loc[idx, 'CasinoManufacturerId'] = parts[0].strip()
        df_casinomanufacturers.loc[idx, 'CasinoManufacturerName'] = parts[1].strip()
        df_casinomanufacturers.loc[idx, 'FromDate'] = parts[2].strip()
        df_casinomanufacturers.loc[idx, 'ToDate'] = parts[3].strip() if parts[3].strip() else None
        df_casinomanufacturers.loc[idx, 'LatestFlag'] = parts[4].strip()

# Data Cleaning

1. Drop duplicates
2. Check the data types and missing values of each column (more analytical in roadmap)
3. Ensure you always use the latest manufacturer record (in df_casinomanufacturers)
4. Convert Data Types and fill NAs
5. Perform age calculation 
6. Create AgeGroup Column
7. Apply currency conversion for GGR (convert the GGR and Returns from NZD to EUR) 

Decide to fill the missing values of the column "ToDate" with "Present" since we can make the assumptions that the reason why this column is null is because this casino manufacturer continues to to cooperate.

In [2]:
# 1. Drop Duplicates from all dataframes

def remove_duplicates():
    dataframes = {'casinodaily': df_casinodaily,
        'casinomanufacturers': df_casinomanufacturers,
        'casinoproviders': df_casinoproviders,
        'currencyrates': df_currencyrates,
        'users': df_users}
    for name, df in dataframes.items():
        df.drop_duplicates(inplace=True)
remove_duplicates()


# -----------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------

# df_casinodaily
# 4. Convert Data Types
df_casinodaily['Date'] = pd.to_datetime(df_casinodaily['Date'])

# -----------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------

# df_casinomanufacturers
# 4. Convert Data Types
df_casinomanufacturers['CasinoManufacturerId'] = pd.to_numeric(df_casinomanufacturers['CasinoManufacturerId'], errors='coerce')
df_casinomanufacturers['LatestFlag'] = pd.to_numeric(df_casinomanufacturers['LatestFlag'], errors='coerce')
df_casinomanufacturers['FromDate'] = pd.to_datetime(df_casinomanufacturers['FromDate'])
df_casinomanufacturers['ToDate'] = pd.to_datetime(df_casinomanufacturers['ToDate'])
# 3. Ensure you always use the latest manufacturer record
df_casinomanufacturers = df_casinomanufacturers[df_casinomanufacturers['LatestFlag'] == 1].reset_index(drop=True)
# 4. Fill nas of ToDate Column with "Present"
df_casinomanufacturers['ToDate'] = df_casinomanufacturers['ToDate'].fillna('Present')

# -----------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------


#  df_currencyrates
#  4 & 7 Convert Data Types & Apply currency conversion for GGR

#   The df dataframe will present the correct Returns & GGR based on EUR. When the transaction is in another 
#   currency it will make the conversion and replace the original values. In the cases where Returns & GGR are
#   already in EUR it will stay same 

df_currencyrates['Date'] = pd.to_datetime(df_currencyrates['Date'])
df = pd.merge(
    df_casinodaily,
    df_currencyrates,
    left_on=['Date', 'CurrencyId'],
    right_on=['Date', 'ToCurrencyId'],
    how='left')
# Convert non-EUR currencies to EUR where rates  or keep the original one which is in EUR
df['GGR'] = np.where(
    df['EuroRate'].notna(),  
    df['GGR'] * df['EuroRate'],  # convert to EUR
    df['GGR'])  # Keep original (assuming its EUR)

df['Returns'] = np.where(
    df['EuroRate'].notna(),
    df['Returns'] * df['EuroRate'],
    df['Returns']
)
# Drop unecessary columns
df.drop(columns=['FromCurrencyId', 'ToCurrencyId', 'ToCurrencySysname',], inplace=True, errors='ignore')

# -----------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------

# df_users
# 4 & 5 & 6  Convert Data Types & Perform age calculation & Create AgeGroup Column
df_users['BirthDate'] = pd.to_datetime(df_users['BirthDate'], errors='coerce')
today = date.today()
df_users['Age'] = (pd.Timestamp(today) - df_users['BirthDate']).dt.days // 365
age_bins = [0, 18, 26, 32, 40, 50, 150]
age_labels = ['Under 18', '21-26', '27-32', '33-40', '41-50', '50+']
def assign_age_group(age):
    if age < 18:
        return 'Under 18'
    elif age >= 21 and age <= 26:
        return '21-26'
    elif age >= 27 and age <= 32:
        return '27-32'
    elif age >= 33 and age <= 40:
        return '33-40'
    elif age >= 41 and age <= 50:
        return '41-50'
    elif age > 50:
        return '50+'
    else:
        return 'Other'  # for ages 18-20
df_users['AgeGroup'] = df_users['Age'].apply(assign_age_group)

# Aggregate the metrics 

All the preprocessed datasets are now combined into a single dataframe with the columns that was asked for.

In [3]:
df_merged = df.merge(
    df_users[['user_id', 'Country', 'Sex', 'AgeGroup', 'VIPStatus']],
    how='left',
    left_on='UserID',
    right_on='user_id')
df_merged = df_merged.drop(columns='user_id')


df_merged = df_merged.merge(
        df_casinomanufacturers[['CasinoManufacturerId', 'CasinoManufacturerName']], 
        on='CasinoManufacturerId', 
        how='left')

df_merged = df_merged.merge(
        df_casinoproviders[['CasinoProviderId', 'CasinoProviderName']], 
        on='CasinoProviderId', 
        how='left')

# -----------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------

final_df = df_merged[["Date", "Country", "Sex", "AgeGroup", "VIPStatus", 
                     "CasinoManufacturerName", "CasinoProviderName","GGR","Returns"]]

final_df

,Date,Country,Sex,AgeGroup,VIPStatus,CasinoManufacturerName,CasinoProviderName,GGR,Returns
0,2025-03-10,NZD,M,27-32,Bronz E,Hacksaw,Nyx,17.133307,9.161142
1,2025-03-10,NZD,M,27-32,Bronz E,Hacksaw,LightAndWonder,17.133307,9.161142
2,2025-03-10,NZD,F,33-40,POTENTIAL,Micro Gaming,MicroGaming,37.283606,10.647367
3,2025-03-10,NZD,F,33-40,POTENTIAL,Micro Gaming,GamesGlobal,37.283606,10.647367
4,2025-03-10,NZD,F,41-50,POTENTIAL,PragmaticPlay,PragmaticPlay,0.540689,0.907531
...,...,...,...,...,...,...,...,...,...
84169,2025-03-31,NaN,NaN,NaN,NaN,PragmaticPlay,PragmaticPlay,0.820806,2391.221376
84170,2025-04-04,NaN,NaN,NaN,NaN,PlayNGo,PlayNGo,0.477588,1.202229
84171,2025-04-19,NaN,NaN,NaN,NaN,PlayNGo,PlayNGo,0.281606,2.255437
84172,2025-04-19,NaN,NaN,NaN,NaN,PlayNGo,PlayNGo,0.331330,2.662155
